In [5]:
import pandas as pd
import numpy as np
import pathlib as pl
import os
import requests

# Get ASEC data from CPS API

## Variables to Retrieve

In [13]:
ASEC_VARIABLES = [
    'A_AGE',
    'A_SEX',
    'A_HGA',
    'PRDTRACE',
    'PEHSPNON',
    'PRDISFLG',
    'PRCITSHP',
    'MARSUPWT',
    'A_LFSR',
    'HRCHECK',
    'A_CLSWKR',
    'CLWK',
    'PEIO1COW',
    'WEMOCG',
    'A_DTOCC',
    'WEMIND',
    'A_MJIND',
    'LJCW',
    'PEARNVAL',
    'HTOTVAL',
    'A_MARITL',
    # 'HCOV',  # health insurance coverage last year
    'FPERSONS',  # number of persons in family
    'FRELU18',  # number of persons in family under 18
]

## Get single year of ASEC data

In [24]:
# this function will retrieve an entire year of ASEC data
def get_asec_year_df(api_key, asec_variables, year):
    base_url = f'https://api.census.gov/data/{year}/cps/asec/mar'
    get_vars = ','.join(asec_variables)
    url = f'{base_url}?get={get_vars}&key={api_key}'
    response = requests.get(url)
    
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    
    df = df.rename(columns={x: x.lower() for x in df.columns})
    df.loc[:, 'year'] = year
    
    for var in df.columns:
        if var == 'marsupwt':
            df[var] = df[var].astype(np.float64)
        else:
            df[var] = df[var].astype(np.int64)
            
    return df

# I've saved my API key in a local file
with open('c:/users/calvi/coding/census_api_key.txt', 'r') as f:
    api_key = f.readline().strip()

asec_2024 = get_asec_year_df(api_key, ASEC_VARIABLES, 2024)
asec_2024.head()

,a_age,a_sex,a_hga,prdtrace,pehspnon,prdisflg,prcitshp,marsupwt,a_lfsr,hrcheck,...,a_dtocc,wemind,a_mjind,ljcw,pearnval,htotval,a_maritl,fpersons,frelu18,year
0,68,2,39,1,2,2,1,1061.89,1,2,...,17,4,10,1,55000,262579,1,2,0,2024
1,68,1,43,1,2,2,1,1061.89,1,2,...,16,5,5,1,24000,262579,1,2,0,2024
2,58,2,44,1,2,1,1,4788.10,1,2,...,8,10,10,4,95000,96802,5,1,0,2024
3,71,2,43,1,2,2,1,5180.75,7,0,...,0,15,0,0,0,104000,1,2,0,2024
4,70,1,46,1,2,2,1,5180.75,7,2,...,0,10,0,1,55000,104000,1,2,0,2024


In [17]:
asec_df = pd.DataFrame()
base_year = 2024
start_year = base_year - 9
for year in range(start_year, base_year + 1):
    print(f'Processing year {year}')
    df_year = get_asec_year_df(api_key, ASEC_VARIABLES, year)
    asec_df = pd.concat([
        asec_df,
        df_year
    ])

asec_df.head()

Processing year 2015
Processing year 2016
Processing year 2017
Processing year 2018
Processing year 2019
Processing year 2020
Processing year 2021
Processing year 2022


## Save raw ASEC data to parquet

In [19]:
parquets_dir = pl.Path('model.ipynb').resolve().parent/'parquets'
if not os.path.exists(str(parquets_dir)):
    os.makedirs(str(parquets_dir))
parquet_file_path = parquets_dir/f'asec_{start_year}to{base_year}.parquet'

asec_df.to_parquet(parquet_file_path)

# Transform ASEC data to prepare for modeling

In [50]:
parquets_dir = pl.Path('model.ipynb').resolve().parent/'parquets'
parquet_file_path = parquets_dir/f'asec_{start_year}to{base_year}.parquet'
asec = pd.read_parquet(parquet_file_path)

## Remove rows

Starting with 1,695,000 rows

In [51]:
print(len(asec))

1695242


In [52]:
# ! filter with WKSWORK to exclude people who barely worked?

# only include people who worked in paying jobs last year
asec = asec[asec.ljcw.isin([1,2,3,4,5,6])].reset_index(drop=True)

len(asec)

847026

## Create modeling variables

### Education

In [53]:
def get_edu_var(df):
    df = df.copy()
    bins = [30, 39, 40, 41, 43, 44, 45, np.inf]
    names = ['LTHS', 'HS', 'SCND', 'AD', 'BA', 'MA', 'DOC']
    df['edu'] = pd.cut(df['a_hga'], bins, labels=names, right=False)
    
    return df

asec = get_edu_var(asec)
asec.edu.value_counts()

edu
HS      220119
BA      194849
SCND    145246
AD       90375
MA       85058
LTHS     79707
DOC      31672
Name: count, dtype: int64

### Race

In [54]:
def get_race_var(df):
    df = df.copy()
    df['race'] = 'other'
    df.loc[df.prdtrace == 1, 'race'] = 'white'
    df.loc[df.prdtrace == 2, 'race'] = 'black'
    df.loc[df.prdtrace == 4, 'race'] = 'asian'
    
    return df

asec = get_race_var(asec)
asec.race.value_counts()

race
white    661835
black     94406
asian     56524
other     34261
Name: count, dtype: int64

### Hispanic

In [55]:
def get_hispanic_var(df):
    df = df.copy()
    df['hisp'] = 'not_hispanic'
    df.loc[df.pehspnon==1, 'hisp'] = 'hispanic'
    
    return df

asec = get_hispanic_var(asec)
asec.hisp.value_counts()

hisp
not_hispanic    685960
hispanic        161066
Name: count, dtype: int64

### Sex

In [56]:
def get_sex_var(df):
    df = df.copy()
    df['sex'] = 'female'
    df.loc[df.a_sex==1, 'sex'] = 'male'
    
    return df

asec = get_sex_var(asec)
asec.sex.value_counts()

sex
male      441229
female    405797
Name: count, dtype: int64

### Citizenship

In [57]:
def get_citizenship_var(df):
    df = df.copy()
    bins = [1, 4, 5, np.inf]
    names = ['native', 'naturalized', 'noncitizen']
    df.loc[:, 'citshp'] = pd.cut(df['prcitshp'],
                                 bins=bins, labels=names, right=False, include_lowest=True)
    
    return df

asec = get_citizenship_var(asec)
asec.citshp.value_counts()

citshp
native         700363
noncitizen      76079
naturalized     70584
Name: count, dtype: int64

### Class of worker

In [58]:
def get_cow_var(df):
    df = df.copy()
    bins = [1, 2, 5, 6, np.inf]
    names = ['ws', 'gov', 'seinc', 'seuninc']
    df.loc[:, 'cow'] = pd.cut(df['ljcw'],
                                 bins=bins, labels=names, 
                                 right=False, include_lowest=True)
    
    return df

asec = get_cow_var(asec)
asec.cow.value_counts()

cow
ws         639444
gov        128180
seuninc     50159
seinc       29243
Name: count, dtype: int64

### Full-time/part-time

In [59]:
def get_ftpt_var(df):
    df = df.copy()
    df['ftpt'] = 'ft'
    df.loc[df.hrcheck == 1, 'ftpt'] = 'pt'
    
    return df

asec = get_ftpt_var(asec)
asec.ftpt.value_counts()

ftpt
ft    682731
pt    164295
Name: count, dtype: int64

### Disability

In [61]:
def get_disability_var(df):
    df = df.copy()
    df['disability'] = 'no'
    df.loc[df.prdisflg == 1, 'disability'] = 'yes'
    
    return df

asec = get_disability_var(asec)
asec.disability.value_counts()

disability
no     809534
yes     37492
Name: count, dtype: int64

In [65]:
# ! need to adjust for inflation
def get_log_income_var(df):
    df = df.copy()
    # initially set to 0
    df['log_income'] = 0.0
    # only assign log value to positive values - we will treat negative and 0 income as all having 0 here
    df.loc[df.htotval > 0, 'log_income'] = np.log(df[df.htotval > 0].htotval)
    
    return df

asec = get_log_income_var(asec)
asec.log_income.describe()

count    847026.000000
mean         11.402650
std           0.858329
min           0.000000
25%          10.933107
50%          11.461843
75%          11.931682
max          15.009578
Name: log_income, dtype: float64